<a href="https://colab.research.google.com/github/tkhabia/labour_worthness_detection/blob/master/labour_worthness_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![AIcrowd-Logo](https://raw.githubusercontent.com/AIcrowd/AIcrowd/master/app/assets/images/misc/aicrowd-horizontal.png)

# Baseline for [LABOR Challenge](www.aicrowd.com/challenges/labor) on AIcrowd
#### Author : Faizan Farooq Khan

## Download Necessary Packages 📚

In [1]:
!pip install numpy
!pip install pandas
!pip install scikit-learn

## Download Data
The first step is to download out train test data. We will be training a model on the train data and make predictions on test data. We submit our predictions


In [2]:
#Donwload the datasets
!rm -rf data
!mkdir data 
!wget https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/labor/v0.1/test.csv
!wget https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/labor/v0.1/train.csv
!mv test.csv data/test.csv
!mv train.csv data/train.csv

--2020-07-24 13:56:21--  https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/labor/v0.1/test.csv
Resolving s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)... 130.117.252.17, 130.117.252.12, 130.117.252.16, ...
Connecting to s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)|130.117.252.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 554341 (541K) [text/csv]
Saving to: ‘test.csv’

test.csv            100%[===================>] 541.35K   798KB/s    in 0.7s    

2020-07-24 13:56:23 (798 KB/s) - ‘test.csv’ saved [554341/554341]

--2020-07-24 13:56:25--  https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/labor/v0.1/train.csv
Resolving s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)... 130.117.252.12, 130.117.252.17, 130.117.252.10, ...
Connecting to s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)|130.117.252.12|:443... connected.
HTTP request sent, awaiting response..


## Import packages

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score


## Load Data
- We use pandas 🐼 library to load our data.   
- Pandas loads the data into dataframes and facilitates us to analyse the data.   
- Learn more about it [here](https://www.tutorialspoint.com/python_data_science/python_pandas.htm) 🤓

In [4]:
all_data_path = "data/train.csv" #path where data is stored

In [5]:
all_data = pd.read_csv(all_data_path) #load data in dataframe using pandas

## Visualize the data 👀

In [6]:
all_data.head()

,duration,wage-increase-first-year,wage-increase-second-year,wage-increase-third-year,cost-of-living-adjustment,working-hours,pension,standby-pay,shift-differential,education-allowance,statutory-holidays,vacation,longterm-disability-assistance,contribution-to-dental-plan,bereavement-assistance,contribution-to-health-plan,class
0,3,3.597483,4.000000,5.000000,0,40.000000,2,8.32238,3.0,0,11.0,0,1,1,1,2,1
1,3,3.968619,4.000000,5.100000,1,40.000000,2,2.00000,3.0,0,12.0,1,1,2,1,2,1
2,2,6.328544,5.089680,5.000000,0,35.915468,2,2.00000,4.0,0,12.0,1,1,1,1,2,1
3,2,4.348288,5.336979,5.000000,0,37.651356,2,2.00000,3.0,0,15.0,1,1,2,1,2,1
4,2,3.530789,2.892247,2.029438,0,40.000000,2,2.00000,4.0,0,11.0,1,1,1,1,2,1


We can see the dataset contains 17 columns,where columns 1-16 denotes the information about the labor and the last column tell whether he is in good(1) class or bad(0) classsubcribed the service or not.

## Split Data into Train and Validation 🔪
-  The next step is to think of a way to test how well our model is performing. we cannot use the test data given as it does not contain the data labels for us to verify.    
- The workaround this is to split the given training data into training and validation. Typically validation sets give us an idea of how our model will perform on unforeseen data. it is like holding back a chunk of data while training our model and then using it to for the purpose of testing. it is a standard way to fine-tune hyperparameters in a model.    
- There are multiple ways to split a dataset into validation and training sets. following are two popular ways to go about it, [k-fold](https://machinelearningmastery.com/k-fold-cross-validation/), [leave one out](https://en.wikipedia.org/wiki/Cross-validation_statistics). 🧐
- Validation sets are also used to avoid your model from [overfitting](https://machinelearningmastery.com/overfitting-and-underfitting-with-machine-learning-algorithms/) on the train dataset.

In [7]:
X_train, X_val= train_test_split(all_data, test_size=0.2, random_state=42) 

- We have decided to split the data with 20 % as validation and 80 % as training.  
- To learn more about the train_test_split function [click here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). 🧐  
- This is of course the simplest way to validate your model by simply taking a random chunk of the train set and setting it aside solely for the purpose of testing our train model on unseen data. as mentioned in the previous block, you can experiment 🔬 with and choose more sophisticated techniques and make your model better.

- Now, since we have our data splitted into train and validation sets, we need to get the corresponding labels separated from the data.   
- with this step we are all set move to the next step with a prepared dataset.

In [8]:
X_train,y_train = X_train.iloc[:,:-1],X_train.iloc[:,-1]
X_val,y_val = X_val.iloc[:,:-1],X_val.iloc[:,-1]

In [9]:
feature_name = X_train.columns
print(len(feature_name))

16


In [10]:
total_features = 16
num_feats = 11

#################################################################

def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature
cor_support, cor_feature = cor_selector(X_train, y_train,num_feats)
print(str(len(cor_feature)), 'selected features')


###############################
#che square 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
X_norm = MinMaxScaler().fit_transform(X_train)
chi_selector = SelectKBest(chi2, k=num_feats)
chi_selector.fit(X_norm, y_train)
chi_support = chi_selector.get_support()
chi_feature = X_train.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')
#################################################
#recursive feature elimination
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=num_feats, step=10, verbose=5)
rfe_selector.fit(X_norm, y_train)
rfe_support = rfe_selector.get_support()
rfe_feature = X_train.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')
##################################################
#LASSO SELECT FROM MODEL
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2"), max_features=num_feats)
embeded_lr_selector.fit(X_norm, y_train)

embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X_train.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')
###################################################
#Tree-based: SelectFromModel
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=num_feats)
embeded_rf_selector.fit(X_train, y_train)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X_train.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')
##########################################3##########
#LightGBM

from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, max_features=num_feats)
embeded_lgb_selector.fit(X_train, y_train)

embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = X_train.loc[:,embeded_lgb_support].columns.tolist()
print(str(len(embeded_lgb_feature)), 'selected features')

#######################################
## all combined 

feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,
                                    'Random Forest':embeded_rf_support, 'LightGBM':embeded_lgb_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(total_features)


11 selected features
11 selected features
Fitting estimator with 16 features.
11 selected features
8 selected features
6 selected features
5 selected features


,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,LightGBM,Total
1,wage-increase-first-year,True,True,True,True,True,True,6
2,statutory-holidays,True,True,True,True,True,False,5
3,pension,True,True,True,True,True,False,5
4,longterm-disability-assistance,True,True,True,True,True,False,5
5,contribution-to-dental-plan,True,True,True,True,True,False,5
6,working-hours,True,False,True,True,False,True,4
7,wage-increase-third-year,True,False,True,True,False,True,4
8,wage-increase-second-year,True,True,True,False,False,True,4
9,shift-differential,False,False,True,True,False,True,3
10,contribution-to-health-plan,True,True,False,False,True,False,3


# TRAINING PHASE 🏋️

## Define the Model

- We have fixed our data and now we are ready to train our model.   

- There are a ton of classifiers to choose from some being [Logistic Regression](https://towardsdatascience.com/logistic-regression-detailed-overview-46c4da4303bc), [SVM](https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47), [Random Forests](https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47), [Decision Trees](https://towardsdatascience.com/decision-trees-in-machine-learning-641b9c4e8052), etc.🧐         

- Remember that there are no hard-laid rules here. you can mix and match classifiers, it is advisable to read up on the numerous techniques and choose the best fit for your solution , experimentation is the key.     
   
- A good model does not depend solely on the classifier but also on the features you choose. So make sure to analyse and understand your data well and move forward with a clear view of the problem at hand.  you can gain important insight from [here](https://towardsdatascience.com/the-5-feature-selection-algorithms-every-data-scientist-need-to-know-3a6b566efd2).🧐         

In [11]:
# def drop_features(X , feature_selection_df):
#   for i in reversed(range (10 , 17)):
#     X.drop([str(feature_selection_df['Feature'][i])] , axis=1)
#     print(feature_selection_df['Feature'][i])
#   print(X.columns)
#   return X
X_train_m  = X_train
for i in range(10 , 17):
  X_train_m = X_train_m.drop(feature_selection_df['Feature'][i] , axis=1)

# X_train_m = drop_features(X_train , feature_selection_df)
X_train_m.head()

,wage-increase-first-year,wage-increase-second-year,wage-increase-third-year,working-hours,pension,shift-differential,statutory-holidays,longterm-disability-assistance,contribution-to-dental-plan
6029,6.828763,4.511225,5.0,32.444148,2,4.000000,12.0,1,1
2937,2.045057,4.000000,5.0,40.000000,2,10.450791,11.0,1,1
16186,3.330269,3.507451,5.0,33.365945,2,-2.900453,11.0,1,0
13762,2.048843,4.000000,5.0,40.000000,2,3.000000,13.0,0,1
21264,3.691248,4.000000,5.0,37.956569,2,3.000000,15.0,1,1


In [18]:
# classifier = SVC(gamma='auto',max_iter=5)
classifier = MLPClassifier(hidden_layer_sizes=(129,36,), max_iter=750 , learning_rate='adaptive')
#from sklearn.linear_model import LogisticRegression
# classifier = LogisticRegression()

- To start you off, We have used a basic [Support Vector Machines](https://scikit-learn.org/stable/modules/svm.html#classification) classifier here.    
- But you can tune parameters and increase the performance. To see the list of parameters visit [here](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).   
- Do keep in mind there exist sophisticated techniques for everything, the key as quoted earlier is to search them and experiment to fit your implementation.

To read more about other sklearn classifiers visit [here 🧐](https://scikit-learn.org/stable/supervised_learning.html). Try and use other classifiers to see how the performance of your model changes. Try using [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) or [MLP](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) and compare how the performance changes.

## Train the classifier

In [19]:
classifier.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(129, 36), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=750,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

Got a warning! Dont worry, its just beacuse the number of iteration is very less(defined in the classifier in the above cell).Increase the number of iterations and see if the warning vanishes and also see how the performance changes.Do remember increasing iterations also increases the running time.( Hint: max_iter=500)

# Validation Phase 🤔
Wonder how well your model learned! Lets check it.

## Predict on Validation

Now we predict using our trained model on the validation set we created and evaluate our model on unforeseen data.

In [20]:
y_pred = classifier.predict(X_val)

## Evaluate the Performance

- We have used basic metrics to quantify the performance of our model.  
- This is a crucial step, you should reason out the metrics and take hints to improve aspects of your model.
- Do read up on the meaning and use of different metrics. there exist more metrics and measures, you should learn to use them correctly with respect to the solution,dataset and other factors. 
- [F1 score](https://en.wikipedia.org/wiki/F1_score) are the metrics for this challenge

In [21]:
precision = precision_score(y_val,y_pred,average='micro')
recall = recall_score(y_val,y_pred,average='micro')
accuracy = accuracy_score(y_val,y_pred)
f1 = f1_score(y_val,y_pred,average='macro')

In [22]:
print("Accuracy of the model is :" ,accuracy)
print("Recall of the model is :" ,recall)
print("Precision of the model is :" ,precision)
print("F1 score of the model is :" ,f1)

Accuracy of the model is : 0.95828125
Recall of the model is : 0.95828125
Precision of the model is : 0.95828125
F1 score of the model is : 0.9536298218859263


# Testing Phase 😅

We are almost done. We trained and validated on the training data. Now its the time to predict on test set and make a submission.

## Load Test Set

Load the test data on which final submission is to be made.

In [ ]:
final_test_path = "data/test.csv"
final_test = pd.read_csv(final_test_path)

## Predict Test Set
Time for the moment of truth! Predict on test set and time to make the submission.

In [ ]:
submission = classifier.predict(final_test)

## Save the prediction to csv

In [ ]:
#change the header according to the submission guidelines

In [ ]:
submission = pd.DataFrame(submission)
submission.to_csv('/tmp/submission.csv',header=['class'],index=False)

🚧 Note :    
- Do take a look at the submission format.   
- The submission file should contain a header.   
- Follow all submission guidelines strictly to avoid inconvenience.

## To download the generated csv in colab run the below command

In [ ]:
try:
  from google.colab import files
  files.download('submission.csv')
except ImportError as e:
  print("Only for Collab") 

### Well Done! 👍 We are all set to make a submission and see your name on leaderborad. Let navigate to [challenge page](https://www.aicrowd.com/challenges/labor) and make one.